# Study Timetable thoughts:

I have to study the following subjects the upcoming semester:

- Fundamentals of Logistics (FoL) with S1, S2 , R1, R2
- Technical fundamentals (TeFun) with S1, S2 , R1, R2
- Fundamentals of Computer Science (FoCSN) with S1, S2 , R1, R2
- Introduction to Scientific Working (SW) with S1, R1
- Technical Logistic System (TLS) with S1, S2 , R1, R2
- Planning Logistic System & Processes (PLS&P) with S1, S2 , R1, R2
- Traffic Planning & Engineering (TrE) with S1, S2 , R1, R2
- Data Management (DM) with S1, S2 , R1
- Linear Programming (LP) with S1, S2 , R1, R2
- Legal requirements and international regulations (LRIR) with S1, S2 , R1, R2

Where S1,S2 are Study sessions 1 and 2 respectively. R1 and R1 are the respective review sessions. Each session would be a 2hr slot.

Goal is to create a timetable, keeping in mind each 11 subjects consist of S1, S2, R1 and R2. So how do i know is it possible to create a combination? because 6x7 is 42 and 11x4 is 44, that means two subjects needs only S1 and R1, it can be Scientific Writing (SW) and Legal requirements and international regulations(LRIR). Also Data Management (DM) needs only S1, S2, R1. 

Now I can find a solution and program it. for a 6rows and 7 column table with proper reinforcement learning or recall ability (not more than 3 days from previous study) included for each subject.

Note: The reinforcement learning ability in a timetable context means ensuring that study sessions and their respective review sessions are spaced in a way that is conducive to memory retention. This is commonly known as spaced repetition, which is a learning technique that incorporates increasing intervals of time between subsequent reviews of previously learned material.

## Simple plan without spaced repetition:

In [1]:
import pulp

# Define the problem
prob = pulp.LpProblem("Timetable", pulp.LpMinimize)

# Create a dictionary for the number of sessions per subject
sessions_per_subject = {
    'FoL': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'TeFun': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'FoCSN': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'SW': {'S1': 1, 'R1': 1},
    'TLS': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'PLS&P': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'TrE': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'DM': {'S1': 1, 'S2': 1, 'R1': 1},
    'LP': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
    'LRIR': {'S1': 1, 'R1': 1}
}

# Create a variable for each session slot for each subject
variables = {}
for subject in sessions_per_subject:
    for session_type in ['S1', 'S2', 'R1', 'R2']:
        for day in range(7):
            for slot in range(6):
                var_name = f"{subject}_{session_type}_{day}_{slot}"
                variables[var_name] = pulp.LpVariable(var_name, cat='Binary')

# The objective function can be a dummy one since we're not optimizing any particular value
prob += 0

# Add constraints for each subject
for subject, num_sessions in sessions_per_subject.items():
    # Constraint: The total number of sessions per subject must match the requirement
    prob += pulp.lpSum([variables[f"{subject}_{session_type}_{day}_{slot}"]
                        for session_type in ['S1', 'S2', 'R1', 'R2']
                        for day in range(7)
                        for slot in range(6)]) == num_sessions

# Add constraints for each slot
for day in range(7):
    for slot in range(6):
        # Constraint: No slot can have more than one session
        prob += pulp.lpSum([variables[f"{subject}_{session_type}_{day}_{slot}"]
                            for subject in sessions_per_subject
                            for session_type in ['S1', 'S2', 'R1', 'R2']]) <= 1

# Solve the problem
prob.solve()

# Output the solution
for variable in prob.variables():
    if variable.varValue == 1:
        print(variable.name)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/codespace/.python/current/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/23ec6c7bde904aada5811dbe277930d8-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/23ec6c7bde904aada5811dbe277930d8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 57 COLUMNS
At line 6779 RHS
At line 6832 BOUNDS
At line 8514 ENDATA
Problem MODEL has 52 rows, 1681 columns and 3360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0008I 42 inequality constraints converted to equality constraints
Cgl0005I 42 SOS with 462 members
Cgl0004I processed model has 52 rows, 462 columns (462 integer (462 of which binary)) and 882 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0
Cbc0038I Before mini branch and bound, 420 integer

In [2]:
# Create a map for day indices to day names
day_map = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday'}

# Initialize an empty timetable
timetable = {day: ['-' for _ in range(6)] for day in day_map.values()}

# Process the solution and fill in the timetable
for variable in prob.variables():
    if variable.varValue == 1:
        # Extract information from the variable name
        parts = variable.name.split('_')
        subject = parts[0]
        session_type = parts[1]
        day = int(parts[2])
        slot = int(parts[3])
        
        # Convert the numerical day to the actual day of the week
        day_name = day_map[day]
        
        # Fill in the timetable
        timetable[day_name][slot] = f"{subject} {session_type}"

# Print the timetable
for day in day_map.values():
    print(f"{day}: {timetable[day]}")


Sunday: ['TeFun R2', 'TLS S1', 'FoCSN R2', 'DM S2', 'TLS S1', 'TrE S2']
Monday: ['PLS&P S1', 'TLS R1', 'FoCSN R1', 'DM R2', '-', 'SW S2']
Tuesday: ['FoL R1', 'TrE S1', 'LRIR S1', 'LP R2', 'TLS S1', '-']
Wednesday: ['FoL R1', 'FoCSN R1', '-', 'PLS&P R1', 'TrE R2', 'TeFun S2']
Thursday: ['LP S2', 'LP S2', 'LP S2', 'TrE S1', 'DM S2', 'FoL S1']
Friday: ['LRIR S2', '-', '-', 'PLS&P R2', 'TeFun R1', 'SW S1']
Saturday: ['-', 'PLS&P R1', '-', 'TeFun R2', 'FoL R2', 'FoCSN S2']


# Spaced Repetition

In [3]:
import pulp

# Define the problem
prob = pulp.LpProblem("Timetable", pulp.LpMinimize)

#sessions_needed = {
#    'FoL': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TeFun': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'FoCSN': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'SW': {'S1': 1, 'R1': 1}, # SW has only 2 sessions
#    'TLS': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'PLS&P': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TrE': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'DM': {'S1': 1, 'S2': 1, 'R1': 1}, # DM has only 3 sessions
#    'LP': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'LRIR': {'S1': 1, 'R1': 1} # LRIR has only 2 sessions
#}

sessions_needed = {
    'FoL': {'S1': 1, 'R1': 1},
    'TeFun': {'S1': 1, 'R1': 1},
    'FoCSN': {'S1': 1, 'R1': 1},
    'SW': {'S1': 1, 'R1': 1}, # SW has only 2 sessions
    'TLS': {'S1': 1, 'R1': 1},
    'PLS&P': {'S1': 1, 'R1': 1},
    'TrE': {'S1': 1, 'R1': 1},
    'DM': {'S1': 1, 'R1': 1}, # DM has only 3 sessions
    'LP': {'S1': 1, 'R1': 1},
    'LRIR': {'S1': 1, 'R1': 1} # LRIR has only 2 sessions
}

# Days of the week
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Create variables for each subject, session type, day, and slot
variables = {}
for subject, sessions in sessions_needed.items():
    for session_type in ['S1', 'S2', 'R1', 'R2']:
        if session_type in sessions:
            for day in days:
                for slot in range(6):
                    var_name = f'{subject}_{session_type}_{day}_{slot}'
                    variables[var_name] = pulp.LpVariable(var_name, cat='Binary')

# Add constraints for the number of sessions needed for each subject
for subject, session_types in sessions_needed.items():
    for session_type, count in session_types.items():
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for day in days
                       for slot in range(6)) == count
        )

# Ensure no overlapping sessions in each slot
for day in days:
    for slot in range(6):
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for subject in sessions_needed
                       for session_type in sessions_needed[subject]) <= 1
        )

# Spaced repetition constraints
for subject in sessions_needed.keys():
    for index, day in enumerate(days):
        for slot in range(6):
            # Study session S1 must occur before S2 and R1
            if 'S1' in sessions_needed[subject] and 'S2' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_S2_{future_day}_{slot}"]
                                   for future_day in days[index+1:])
            if 'S1' in sessions_needed[subject] and 'R1' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R1_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R1 should be within 3 days after S1
            # Review session R1 must occur before R2
            if 'R1' in sessions_needed[subject] and 'R2' in sessions_needed[subject]:
                prob += variables[f"{subject}_R1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R2_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R2 should be within 3 days after R1

# Solve the problem
status = prob.solve()

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Print the timetable in a readable format
    for day in days:
        for slot in range(6):
            slot_sessions = []
            for subject in sessions_needed.keys():
                for session_type in sessions_needed[subject]:
                    if pulp.value(variables[f"{subject}_{session_type}_{day}_{slot}"]) == 1:
                        slot_sessions.append(f"{subject} {session_type}")
            print(f"{day} Slot {slot}: {slot_sessions}")
else:
    print("No feasible solution found.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/codespace/.python/current/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/fec88b9dae884cea9c2cbd1edb2ad512-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/fec88b9dae884cea9c2cbd1edb2ad512-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 487 COLUMNS
At line 5169 RHS
At line 5652 BOUNDS
At line 6494 ENDATA
Problem MODEL has 482 rows, 841 columns and 3000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 60 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 120 strengthened rows, 0 substitutions
Cgl0004I processed model has 422 rows, 780 columns (780 integer (780 of which binary)) and 2940 elements
Cbc0045I No integer variables out of 780 objects (780 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y 

In [4]:
import pandas as pd

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Initialize the timetable with empty strings
    timetable_data = {day: ['' for _ in range(6)] for day in days}
    
    # Fill the timetable with the scheduled sessions
    for variable in prob.variables():
        if variable.varValue == 1:
            parts = variable.name.split('_')
            subject = parts[0]
            session_type = parts[1]
            day_name = parts[2]  # Directly get the day name
            slot = int(parts[3])  # Index of the slot
            timetable_data[day_name][slot] += f"{subject} {session_type} "

    # Create a DataFrame from the dictionary
    timetable_df = pd.DataFrame.from_dict(timetable_data, orient='index', columns=[f'Slot {i+1}' for i in range(6)])
    
    # Print the DataFrame
    print(timetable_df)
else:
    print("No feasible solution found.")


             Slot 1     Slot 2  Slot 3     Slot 4   Slot 5     Slot 6
Sunday     LRIR S1                                          TeFun S1 
Monday                                  FoCSN S1               LP S1 
Tuesday    LRIR R1   PLS&P S1                                        
Wednesday                       SW S1   FoCSN R1   TrE S1   TeFun R1 
Thursday    TLS S1   PLS&P R1                      FoL S1      LP R1 
Friday      TLS R1      DM S1   SW R1              FoL R1            
Saturday                DM R1                      TrE R1            


In [9]:
import pulp

# Define the problem
prob = pulp.LpProblem("Timetable", pulp.LpMinimize)

#sessions_needed = {
#    'FoL': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TeFun': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'FoCSN': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'SW': {'S1': 1, 'R1': 1}, # SW has only 2 sessions
#    'TLS': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'PLS&P': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TrE': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'DM': {'S1': 1, 'S2': 1, 'R1': 1}, # DM has only 3 sessions
#    'LP': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'LRIR': {'S1': 1, 'R1': 1} # LRIR has only 2 sessions
#}

sessions_needed = {
    'FoL': {'S1': 1, 'R1': 1},
    'TeFun': {'S1': 1, 'R1': 1},
    'FoCSN': {'S1': 1, 'S2': 1, 'R1': 1},
    'SW': {'S1': 1, 'R1': 1}, # SW has only 2 sessions
    'TLS': {'S1': 1, 'R1': 1},
    'PLS&P': {'S1': 1, 'R1': 1},
    'TrE': {'S1': 1, 'R1': 1},
    'DM': {'S1': 1, 'R1': 1}, # DM has only 3 sessions
    'LP': {'S1': 1, 'S2': 1, 'R1': 1},
    'LRIR': {'S1': 1, 'R1': 1} # LRIR has only 2 sessions
}

# Days of the week
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Create variables for each subject, session type, day, and slot
variables = {}
for subject, sessions in sessions_needed.items():
    for session_type in ['S1', 'S2', 'R1', 'R2']:
        if session_type in sessions:
            for day in days:
                for slot in range(6):
                    var_name = f'{subject}_{session_type}_{day}_{slot}'
                    variables[var_name] = pulp.LpVariable(var_name, cat='Binary')

# Add constraints for the number of sessions needed for each subject
for subject, session_types in sessions_needed.items():
    for session_type, count in session_types.items():
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for day in days
                       for slot in range(6)) == count
        )

# Ensure no overlapping sessions in each slot
for day in days:
    for slot in range(6):
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for subject in sessions_needed
                       for session_type in sessions_needed[subject]) <= 1
        )

# Spaced repetition constraints
for subject in sessions_needed.keys():
    for index, day in enumerate(days):
        for slot in range(6):
            # Study session S1 must occur before S2 and R1
            if 'S1' in sessions_needed[subject] and 'S2' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_S2_{future_day}_{slot}"]
                                   for future_day in days[index+1:])
            if 'S1' in sessions_needed[subject] and 'R1' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R1_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R1 should be within 3 days after S1
            # Review session R1 must occur before R2
            if 'R1' in sessions_needed[subject] and 'R2' in sessions_needed[subject]:
                prob += variables[f"{subject}_R1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R2_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R2 should be within 3 days after R1

# Solve the problem
status = prob.solve(pulp.PULP_CBC_CMD(msg=False))  # Add msg=False here

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Print the timetable in a readable format
    for day in days:
        for slot in range(6):
            slot_sessions = []
            for subject in sessions_needed.keys():
                for session_type in sessions_needed[subject]:
                    if pulp.value(variables[f"{subject}_{session_type}_{day}_{slot}"]) == 1:
                        slot_sessions.append(f"{subject} {session_type}")
            #print(f"{day} Slot {slot}: {slot_sessions}")
else:
    print("No feasible solution found.")

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Initialize the timetable with empty strings
    timetable_data = {day: ['' for _ in range(6)] for day in days}
    
    # Fill the timetable with the scheduled sessions
    for variable in prob.variables():
        if variable.varValue == 1:
            parts = variable.name.split('_')
            subject = parts[0]
            session_type = parts[1]
            day_name = parts[2]  # Directly get the day name
            slot = int(parts[3])  # Index of the slot
            timetable_data[day_name][slot] += f"{subject} {session_type} "

    # Create a DataFrame from the dictionary
    timetable_df = pd.DataFrame.from_dict(timetable_data, orient='index', columns=[f'Slot {i+1}' for i in range(6)])
    
    # Print the DataFrame
    print(timetable_df)
else:
    print("No feasible solution found.")

              Slot 1     Slot 2   Slot 3   Slot 4  Slot 5     Slot 6
Sunday                           TLS S1                             
Monday        SW S1   FoCSN S1             LP S1                    
Tuesday               FoCSN R1   TLS R1                             
Wednesday  PLS&P S1                        LP R1           TeFun S1 
Thursday      SW R1   FoCSN S2   TrE S1   FoL S1           TeFun R1 
Friday     PLS&P R1    LRIR S1             LP S2   DM S1            
Saturday               LRIR R1   TrE R1   FoL R1   DM R1            


# Test with 8 Slot

In [14]:
import pulp

# Define the problem
prob = pulp.LpProblem("Timetable", pulp.LpMinimize)

#sessions_needed = {
#    'FoL': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TeFun': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'FoCSN': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'SW': {'S1': 1, 'R1': 1}, # SW has only 2 sessions
#    'TLS': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'PLS&P': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'TrE': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'DM': {'S1': 1, 'S2': 1, 'R1': 1}, # DM has only 3 sessions
#    'LP': {'S1': 1, 'S2': 1, 'R1': 1, 'R2': 1},
#    'LRIR': {'S1': 1, 'R1': 1} # LRIR has only 2 sessions
#}



sessions_needed = {
    # - Fundamentals of Logistics (FoL) with S1, S2 , R1, R2
    'FoL': {'S1': 1,
            'S2': 1,
            'R1': 1},
    # - Technical fundamentals (TeFun) with S1, S2 , R1, R2
    'TeFun': {'S1': 1,
              'R1': 1},
    # - Fundamentals of Computer Science (FoCSN) with S1, S2 , R1, R2
    'FoCSN': {'S1': 1, 
              'S2': 1,
              'R1': 1},
    # - Introduction to Scientific Working (SW) with S1, R1
    'SW': {'S1': 1, 
           'R1': 1}, # SW has only 2 sessions
    # - Technical Logistic System (TLS) with S1, S2 , R1, R2
    'TLS': {'S1': 1, 
            'R1': 1},
    # - Planning Logistic System & Processes (PLS&P) with S1, S2 , R1, R2
    'PLS&P': {'S1': 1,
              'R1': 1},
    # - Traffic Planning & Engineering (TrE) with S1, S2 , R1, R2
    'TrE': {'S1': 1,
            'R1': 1},
    # - Data Management (DM) with S1, S2 , R1
    'DM': {'S1': 1, 
           'R1': 1}, # DM has only 3 sessions
    # - Linear Programming (LP) with S1, S2 , R1, R2
    'LP': {'S1': 1, 
           'S2': 1,
           'R1': 1},
    # - Legal requirements and international regulations (LRIR) with S1, S2 , R1, R2
    'LRIR': {'S1': 1,
             'R1': 1} # LRIR has only 2 sessions
}

num_of_slots = 8

# Days of the week
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Create variables for each subject, session type, day, and slot
variables = {}
for subject, sessions in sessions_needed.items():
    for session_type in ['S1', 'S2', 'R1', 'R2']:
        if session_type in sessions:
            for day in days:
                for slot in range(num_of_slots):
                    var_name = f'{subject}_{session_type}_{day}_{slot}'
                    variables[var_name] = pulp.LpVariable(var_name, cat='Binary')

# Add constraints for the number of sessions needed for each subject
for subject, session_types in sessions_needed.items():
    for session_type, count in session_types.items():
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for day in days
                       for slot in range(num_of_slots)) == count
        )

# Ensure no overlapping sessions in each slot
for day in days:
    for slot in range(num_of_slots):
        prob += (
            pulp.lpSum(variables[f"{subject}_{session_type}_{day}_{slot}"]
                       for subject in sessions_needed
                       for session_type in sessions_needed[subject]) <= 1
        )

# Spaced repetition constraints
for subject in sessions_needed.keys():
    for index, day in enumerate(days):
        for slot in range(num_of_slots):
            # Study session S1 must occur before S2 and R1
            if 'S1' in sessions_needed[subject] and 'S2' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_S2_{future_day}_{slot}"]
                                   for future_day in days[index+1:])
            if 'S1' in sessions_needed[subject] and 'R1' in sessions_needed[subject]:
                prob += variables[f"{subject}_S1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R1_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R1 should be within 3 days after S1
            # Review session R1 must occur before R2
            if 'R1' in sessions_needed[subject] and 'R2' in sessions_needed[subject]:
                prob += variables[f"{subject}_R1_{day}_{slot}"] <= \
                        pulp.lpSum(variables[f"{subject}_R2_{future_day}_{slot}"]
                                   for future_day in days[index+1:index+4])  # R2 should be within 3 days after R1

# Solve the problem
status = prob.solve(pulp.PULP_CBC_CMD(msg=False))  # Add msg=False here

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Print the timetable in a readable format
    for day in days:
        for slot in range(num_of_slots):
            slot_sessions = []
            for subject in sessions_needed.keys():
                for session_type in sessions_needed[subject]:
                    if pulp.value(variables[f"{subject}_{session_type}_{day}_{slot}"]) == 1:
                        slot_sessions.append(f"{subject} {session_type}")
            #print(f"{day} Slot {slot}: {slot_sessions}")
else:
    print("No feasible solution found.")

# Check if a feasible solution exists
if status == pulp.LpStatusOptimal:
    # Initialize the timetable with empty strings
    timetable_data = {day: ['' for _ in range(num_of_slots)] for day in days}
    
    # Fill the timetable with the scheduled sessions
    for variable in prob.variables():
        if variable.varValue == 1:
            parts = variable.name.split('_')
            subject = parts[0]
            session_type = parts[1]
            day_name = parts[2]  # Directly get the day name
            slot = int(parts[3])  # Index of the slot
            timetable_data[day_name][slot] += f"{subject} {session_type} "

    # Create a DataFrame from the dictionary
    timetable_df = pd.DataFrame.from_dict(timetable_data, orient='index', columns=[f'Slot {i+1}' for i in range(num_of_slots)])
    
    # Print the DataFrame
    print(timetable_df)
else:
    print("No feasible solution found.")

           Slot 1   Slot 2     Slot 3     Slot 4 Slot 5  Slot 6 Slot 7  \
Sunday                      PLS&P S1                     DM S1           
Monday             FoL S1                                                
Tuesday    SW S1   FoL R1   PLS&P R1   FoCSN S1          DM R1           
Wednesday                                TLS S1                          
Thursday           FoL S2              FoCSN S2          LP S1           
Friday     SW R1             LRIR S1   FoCSN R1          LP S2           
Saturday                     LRIR R1     TLS R1          LP R1           

              Slot 8  
Sunday                
Monday     TeFun S1   
Tuesday               
Wednesday  TeFun R1   
Thursday              
Friday       TrE S1   
Saturday     TrE R1   
